# 🎨 Image & Video Generation — Stable Diffusion + Gradio

**Runtime: GPU (T4 recommended)**  
Runtime → Change runtime type → T4 GPU

### What this notebook does:
- **Image generation** — Stable Diffusion XL (SDXL) via 🤗 Diffusers
- **Image-to-image** — transform existing images with a prompt
- **Video generation** — AnimateDiff (text → short MP4 clip)
- **Gradio UI** — all features in one polished tabbed interface

In [ ]:
# ── Cell 1: Install dependencies ──────────────────────────────────────────────
!pip install -q \
    diffusers>=0.27.0 \
    transformers>=4.40.0 \
    accelerate>=0.29.0 \
    safetensors>=0.4.0 \
    gradio>=4.26.0 \
    imageio[ffmpeg]>=2.34.0 \
    xformers \
    invisible-watermark \
    opencv-python-headless

print('✅ Dependencies installed')

In [ ]:
# ── Cell 2: Imports & GPU check ───────────────────────────────────────────────
import torch
import gradio as gr
import numpy as np
from PIL import Image
import imageio
import tempfile
import os
import gc
from diffusers import (
    StableDiffusionXLPipeline,
    StableDiffusionXLImg2ImgPipeline,
    AnimateDiffPipeline,
    MotionAdapter,
    EulerDiscreteScheduler,
    DDIMScheduler,
)
from diffusers.utils import export_to_video

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DTYPE  = torch.float16 if DEVICE == 'cuda' else torch.float32

print(f'🖥️  Device : {DEVICE}')
if DEVICE == 'cuda':
    print(f'🎮  GPU    : {torch.cuda.get_device_name(0)}')
    print(f'💾  VRAM   : {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('⚠️  No GPU detected — image generation will be very slow on CPU.')

In [ ]:
# ── Cell 3: Load pipelines (lazy — loaded on first use) ───────────────────────
_txt2img_pipe  = None
_img2img_pipe  = None
_video_pipe    = None

SDXL_MODEL     = 'stabilityai/stable-diffusion-xl-base-1.0'
ANIMDIFF_MODEL = 'guoyww/animatediff-motion-adapter-v1-5-2'
ANIMDIFF_BASE  = 'SG161222/Realistic_Vision_V5.1_noVAE'


def get_txt2img():
    global _txt2img_pipe
    if _txt2img_pipe is None:
        print('⏳ Loading SDXL text-to-image pipeline...')
        _txt2img_pipe = StableDiffusionXLPipeline.from_pretrained(
            SDXL_MODEL,
            torch_dtype=DTYPE,
            use_safetensors=True,
            variant='fp16' if DEVICE == 'cuda' else None,
        ).to(DEVICE)
        if DEVICE == 'cuda':
            _txt2img_pipe.enable_xformers_memory_efficient_attention()
        print('✅ SDXL text-to-image ready')
    return _txt2img_pipe


def get_img2img():
    global _img2img_pipe
    if _img2img_pipe is None:
        print('⏳ Loading SDXL image-to-image pipeline...')
        _img2img_pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
            SDXL_MODEL,
            torch_dtype=DTYPE,
            use_safetensors=True,
            variant='fp16' if DEVICE == 'cuda' else None,
        ).to(DEVICE)
        if DEVICE == 'cuda':
            _img2img_pipe.enable_xformers_memory_efficient_attention()
        print('✅ SDXL image-to-image ready')
    return _img2img_pipe


def get_video():
    global _video_pipe
    if _video_pipe is None:
        print('⏳ Loading AnimateDiff pipeline...')
        adapter = MotionAdapter.from_pretrained(
            ANIMDIFF_MODEL, torch_dtype=DTYPE
        )
        _video_pipe = AnimateDiffPipeline.from_pretrained(
            ANIMDIFF_BASE,
            motion_adapter=adapter,
            torch_dtype=DTYPE,
        ).to(DEVICE)
        _video_pipe.scheduler = DDIMScheduler.from_config(
            _video_pipe.scheduler.config,
            beta_schedule='linear',
            clip_sample=False,
            timestep_spacing='linspace',
            steps_offset=1,
        )
        if DEVICE == 'cuda':
            _video_pipe.enable_vae_slicing()
        print('✅ AnimateDiff ready')
    return _video_pipe


def free_memory():
    gc.collect()
    if DEVICE == 'cuda':
        torch.cuda.empty_cache()

print('✅ Pipeline loaders defined')

In [ ]:
# ── Cell 4: Generation functions ──────────────────────────────────────────────

def generate_image(
    prompt,
    negative_prompt,
    width,
    height,
    steps,
    guidance_scale,
    seed,
):
    """Text → Image using SDXL."""
    if not prompt.strip():
        return None, '⚠️ Please enter a prompt.'
    try:
        generator = torch.Generator(device=DEVICE).manual_seed(int(seed)) if seed >= 0 else None
        pipe = get_txt2img()
        result = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt or None,
            width=int(width),
            height=int(height),
            num_inference_steps=int(steps),
            guidance_scale=float(guidance_scale),
            generator=generator,
        )
        free_memory()
        img = result.images[0]
        info = (f'✅ Generated {img.width}×{img.height}  |  '
                f'Steps: {steps}  |  CFG: {guidance_scale}  |  Seed: {seed}')
        return img, info
    except Exception as e:
        return None, f'❌ Error: {e}'


def generate_img2img(
    init_image,
    prompt,
    negative_prompt,
    strength,
    steps,
    guidance_scale,
    seed,
):
    """Image + Prompt → new Image using SDXL img2img."""
    if init_image is None:
        return None, '⚠️ Please upload an input image.'
    if not prompt.strip():
        return None, '⚠️ Please enter a prompt.'
    try:
        # Resize to nearest 64-multiple
        img = Image.fromarray(init_image).convert('RGB')
        w = (img.width  // 64) * 64
        h = (img.height // 64) * 64
        img = img.resize((max(w, 512), max(h, 512)))

        generator = torch.Generator(device=DEVICE).manual_seed(int(seed)) if seed >= 0 else None
        pipe = get_img2img()
        result = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt or None,
            image=img,
            strength=float(strength),
            num_inference_steps=int(steps),
            guidance_scale=float(guidance_scale),
            generator=generator,
        )
        free_memory()
        out = result.images[0]
        info = (f'✅ Img2Img done  |  Strength: {strength}  |  '
                f'Steps: {steps}  |  CFG: {guidance_scale}')
        return out, info
    except Exception as e:
        return None, f'❌ Error: {e}'


def generate_video(
    prompt,
    negative_prompt,
    num_frames,
    steps,
    guidance_scale,
    fps,
    seed,
):
    """Text → Video (MP4) using AnimateDiff."""
    if not prompt.strip():
        return None, '⚠️ Please enter a prompt.'
    try:
        generator = torch.Generator(device=DEVICE).manual_seed(int(seed)) if seed >= 0 else None
        pipe = get_video()
        output = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt or None,
            num_frames=int(num_frames),
            num_inference_steps=int(steps),
            guidance_scale=float(guidance_scale),
            generator=generator,
        )
        free_memory()
        frames = output.frames[0]  # list of PIL images

        # Save to temp MP4
        tmp = tempfile.NamedTemporaryFile(suffix='.mp4', delete=False)
        export_to_video(frames, tmp.name, fps=int(fps))
        info = (f'✅ Video: {len(frames)} frames @ {fps}fps  |  '
                f'Steps: {steps}  |  CFG: {guidance_scale}')
        return tmp.name, info
    except Exception as e:
        return None, f'❌ Error: {e}'


print('✅ Generation functions defined')

In [ ]:
# ── Cell 5: Gradio UI ─────────────────────────────────────────────────────────

CSS = """
.gradio-container { max-width: 900px !important; margin: auto !important; }
#title { text-align: center; margin-bottom: 8px; }
#title h1 { font-size: 2em; font-weight: 800; }
#title p  { color: #888; margin-top: 4px; }
.status-box textarea { font-size: 13px !important; }
"""

NEG_DEFAULT = (
    'blurry, low quality, ugly, deformed, watermark, text, '
    'bad anatomy, extra limbs, duplicate'
)

with gr.Blocks(css=CSS, theme=gr.themes.Soft(), title='🎨 AI Generator') as demo:

    gr.HTML('<div id="title"><h1>🎨 AI Image &amp; Video Generator</h1>'
            '<p>SDXL · AnimateDiff · Running on ' + DEVICE.upper() + '</p></div>')

    # ── Tab 1: Text → Image ───────────────────────────────────────────────────
    with gr.Tab('🖼️ Text → Image'):
        with gr.Row():
            with gr.Column(scale=3):
                t2i_prompt = gr.Textbox(
                    label='Prompt',
                    placeholder='A majestic wolf howling at the moon, digital art, 4K, dramatic lighting...',
                    lines=3,
                )
                t2i_neg = gr.Textbox(
                    label='Negative Prompt',
                    value=NEG_DEFAULT,
                    lines=2,
                )
                with gr.Row():
                    t2i_w      = gr.Slider(512, 1024, value=1024, step=64,  label='Width')
                    t2i_h      = gr.Slider(512, 1024, value=1024, step=64,  label='Height')
                with gr.Row():
                    t2i_steps  = gr.Slider(10,  50,   value=30,   step=1,   label='Steps')
                    t2i_cfg    = gr.Slider(1,   15,   value=7.5,  step=0.5, label='Guidance Scale')
                with gr.Row():
                    t2i_seed   = gr.Number(value=42, label='Seed (-1 = random)')
                t2i_btn = gr.Button('✨ Generate Image', variant='primary')

            with gr.Column(scale=3):
                t2i_out    = gr.Image(label='Output', type='pil', height=420)
                t2i_status = gr.Textbox(label='Status', interactive=False, elem_classes='status-box')

        gr.Examples(
            examples=[
                ['A futuristic cyberpunk city at night, neon lights, rain reflections, cinematic', NEG_DEFAULT, 1024, 1024, 30, 7.5, 42],
                ['Portrait of an astronaut in a flower field, golden hour, photorealistic, 8K', NEG_DEFAULT, 1024, 1024, 30, 7.5, 7],
                ['Underwater coral reef with tropical fish, vivid colors, National Geographic', NEG_DEFAULT, 1024, 768, 25, 7.0, 99],
            ],
            inputs=[t2i_prompt, t2i_neg, t2i_w, t2i_h, t2i_steps, t2i_cfg, t2i_seed],
            label='Quick Examples',
        )

        t2i_btn.click(
            fn=generate_image,
            inputs=[t2i_prompt, t2i_neg, t2i_w, t2i_h, t2i_steps, t2i_cfg, t2i_seed],
            outputs=[t2i_out, t2i_status],
        )

    # ── Tab 2: Image → Image ──────────────────────────────────────────────────
    with gr.Tab('🔄 Image → Image'):
        with gr.Row():
            with gr.Column(scale=3):
                i2i_input  = gr.Image(label='Input Image', type='numpy', height=240)
                i2i_prompt = gr.Textbox(
                    label='Prompt',
                    placeholder='Transform this into a watercolor painting...',
                    lines=2,
                )
                i2i_neg    = gr.Textbox(label='Negative Prompt', value=NEG_DEFAULT, lines=2)
                with gr.Row():
                    i2i_strength = gr.Slider(0.1, 1.0, value=0.75, step=0.05, label='Strength (how much to change)')
                    i2i_steps    = gr.Slider(10,  50,  value=30,   step=1,    label='Steps')
                with gr.Row():
                    i2i_cfg  = gr.Slider(1, 15, value=7.5, step=0.5, label='Guidance Scale')
                    i2i_seed = gr.Number(value=42, label='Seed (-1 = random)')
                i2i_btn = gr.Button('🔄 Transform Image', variant='primary')

            with gr.Column(scale=3):
                i2i_out    = gr.Image(label='Output', type='pil', height=420)
                i2i_status = gr.Textbox(label='Status', interactive=False, elem_classes='status-box')

        i2i_btn.click(
            fn=generate_img2img,
            inputs=[i2i_input, i2i_prompt, i2i_neg, i2i_strength, i2i_steps, i2i_cfg, i2i_seed],
            outputs=[i2i_out, i2i_status],
        )

    # ── Tab 3: Text → Video ───────────────────────────────────────────────────
    with gr.Tab('🎬 Text → Video'):
        gr.Markdown(
            '> **AnimateDiff** generates short animated clips (~2–4 seconds). '
            'Uses a separate SD 1.5-based model for best motion quality. '
            'Requires ~6 GB VRAM — reduce frames if you hit OOM.'
        )
        with gr.Row():
            with gr.Column(scale=3):
                vid_prompt = gr.Textbox(
                    label='Prompt',
                    placeholder='A golden retriever running through a meadow, slow motion, cinematic...',
                    lines=3,
                )
                vid_neg   = gr.Textbox(label='Negative Prompt', value=NEG_DEFAULT, lines=2)
                with gr.Row():
                    vid_frames = gr.Slider(8,  32,  value=16,  step=4,   label='Frames')
                    vid_fps    = gr.Slider(4,  16,  value=8,   step=1,   label='FPS')
                with gr.Row():
                    vid_steps  = gr.Slider(10, 50,  value=25,  step=1,   label='Steps')
                    vid_cfg    = gr.Slider(1,  15,  value=7.5, step=0.5, label='Guidance Scale')
                vid_seed  = gr.Number(value=42, label='Seed (-1 = random)')
                vid_btn   = gr.Button('🎬 Generate Video', variant='primary')

            with gr.Column(scale=3):
                vid_out    = gr.Video(label='Output Video', height=380)
                vid_status = gr.Textbox(label='Status', interactive=False, elem_classes='status-box')

        gr.Examples(
            examples=[
                ['A campfire burning in a forest at night, sparks flying, cozy atmosphere', NEG_DEFAULT, 16, 8, 25, 7.5, 42],
                ['Ocean waves crashing on a rocky shore, dramatic, cinematic', NEG_DEFAULT, 16, 8, 25, 7.5, 0],
                ['Time-lapse of clouds moving over mountains, peaceful, 4K', NEG_DEFAULT, 24, 8, 30, 7.0, 5],
            ],
            inputs=[vid_prompt, vid_neg, vid_frames, vid_fps, vid_steps, vid_cfg, vid_seed],
            label='Quick Examples',
        )

        vid_btn.click(
            fn=generate_video,
            inputs=[vid_prompt, vid_neg, vid_frames, vid_steps, vid_cfg, vid_fps, vid_seed],
            outputs=[vid_out, vid_status],
        )

    # ── Tab 4: Model Info ─────────────────────────────────────────────────────
    with gr.Tab('ℹ️ Info'):
        gr.Markdown(f"""
## Models
| Task | Model | Notes |
|------|-------|-------|
| Text→Image | stabilityai/stable-diffusion-xl-base-1.0 | SDXL, 6.9B params |
| Image→Image | stabilityai/stable-diffusion-xl-base-1.0 | Same model, img2img pipeline |
| Text→Video | SG161222/Realistic_Vision_V5.1_noVAE + AnimateDiff v1.5.2 | SD1.5-based |

## Tips
- **Quality prompts:** add `4K, highly detailed, professional photography, cinematic`
- **Negative prompts:** always include deformity/quality negatives
- **Guidance Scale:** 7–9 for balanced results; higher = more prompt-adherent
- **Strength (img2img):** 0.3–0.5 for subtle changes, 0.7–0.9 for big transformations
- **Video OOM?** Reduce frames from 16 to 8, or restart runtime

## Hardware
- Current device: **{DEVICE.upper()}**
- Recommended: T4 GPU (free Colab tier) or A100 for video
        """)

print('✅ UI built — launching...')
demo.launch(share=True, debug=False)